# Librerías

In [1]:
import h5py
import scanpy as sc
import numpy as np
import pandas as pd

# Lectura de datos
La prueba se realizará únicamente con el dataset de Human Liver.
Se leen los datos crudos y se selecciona el 10% de células con mayor conteo. Esas células se dividen en dos grupos:
- Grupo 1: Pertenece al cluster más grande (cluster número 7)
- Grupo 2: Pertenece a cualquier otro cluster

In [2]:
dataLiver = h5py.File('../Tests-GMM/data/Small_Datasets/' + 'HumanLiver_counts_top5000.h5')
x = np.array(dataLiver['X'])
y = np.array(dataLiver['Y'])

adata = sc.AnnData(x)
adata.obs['Group'] = y

In [3]:
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [4]:
quantile = pd.qcut(adata.obs['total_counts'], 10, labels = False) 
adata = adata[quantile == 9]

In [5]:
adata.obs['Group2'] = (adata.obs.Group == 7.0)

C:\Users\melir\AppData\Local\Temp\ipykernel_21172\289915785.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['Group2'] = (adata.obs.Group == 7.0)


In [6]:
pd.DataFrame(adata.X).to_csv('mejor_representadas_human_liver.csv', index = None)

In [7]:
pd.DataFrame(adata.obs).to_csv('sample_info.csv', index = None)

# Resultado Deseq2

Se seleccionaron los genes que cumplían con las dos condiciones
- p-value menor a 0.05
- LFC mayor a 1 en valor absoluto 

In [8]:
dataLiver.keys()

<KeysViewHDF5 ['X', 'Y', 'top_2000_indx']>

In [9]:
expressed_genes = pd.read_csv('data_deseq/differentially_expressed_genes.csv')
expressed_genes.rename(columns = {'Unnamed: 0': 'gene'}, inplace = True)
expressed_genes['gene'] = expressed_genes['gene'].apply(lambda x: int(x[1:])) 
expressed_genes.shape

(832, 7)

In [10]:
expressed_genes = expressed_genes[expressed_genes.padj < 0.05]

In [11]:
expressed_genes.shape

(831, 7)

In [12]:
expressed_genes

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,26,2.651295,-3.249906,0.146490,-22.185166,4.776620e-109,1.166860e-107
1,38,3.663920,-4.123774,0.133181,-30.963795,1.656849e-210,9.941094e-209
2,39,1.875427,4.190936,0.213160,19.661030,4.651829e-86,8.887853e-85
3,67,1.295050,2.896206,0.182147,15.900350,6.301136e-57,8.831920e-56
4,70,0.349575,2.212793,0.288031,7.682482,1.560360e-14,6.604492e-14
...,...,...,...,...,...,...,...
827,4977,0.214187,2.500875,1.180176,2.119069,3.408460e-02,4.769612e-02
828,4984,0.694997,2.319342,0.198726,11.671064,1.791688e-31,1.567154e-30
829,4988,0.170798,2.061339,0.469957,4.386225,1.153348e-05,2.463742e-05
830,4990,2.012171,-2.665099,0.152485,-17.477812,2.114574e-68,3.363648e-67


# Lectura datos paper

In [13]:
d = h5py.File('data_deseq/HumanLiver_counts_top5000.h5')

In [14]:
genes = np.array(d['gene_name'])

In [15]:
expressed_genes = genes[expressed_genes.gene]

In [16]:
expressed_genes

array([b'ERRFI1', b'MASP2', b'SRM', b'DDOST', b'USP48', b'HSPG2', b'C1QA',
       b'C1QC', b'C1QB', b'TCEA3', b'ID3', b'PNRC2', b'RHCE', b'STMN1',
       b'CD52', b'HMGN2', b'FCN3', b'LAPTM5', b'FAM167B', b'MARCKSL1',
       b'YBX1', b'SLC2A1', b'CDC20', b'NASP', b'TXNDC12', b'GPX7', b'JUN',
       b'ANGPTL3', b'RP11-386I14.4', b'CD53', b'FAM46C', b'RP11-403I13.5',
       b'CH17-373J23.1', b'ANP32E', b'CTSS', b'TCHH', b'S100A4',
       b'S100A16', b'NPR1', b'RAB25', b'PMF1', b'FCRL2', b'CRP', b'FCRL6',
       b'LINC01133', b'SLAMF7', b'LY9', b'ADAMTS4', b'NR1I3', b'FCRLA',
       b'MPC2', b'DNM3', b'CACYBP', b'MRPS14', b'QSOX1', b'RGL1', b'TPR',
       b'RGS13', b'UCHL5', b'CFHR3', b'CFHR1', b'CFHR4', b'CRB1',
       b'LMOD1', b'ADIPOR1', b'CHI3L1', b'FMOD', b'GOLT1A', b'RAB7B',
       b'C4BPA', b'CD55', b'TRAF3IP3', b'LINC00467', b'PTPN14', b'TGFB2',
       b'IARS2', b'DEGS1', b'LEFTY1', b'GJC2', b'FAM89A', b'SH3YL1',
       b'C2orf48', b'PDIA6', b'TMEM214', b'PREB', b'SLC5A6', b'NRBP

# Resultados Deseq2 - ajustes Single Cell

In [20]:
expressed_genes = pd.read_csv('data_deseq/differentially_expressed_genes_sc.csv')
expressed_genes.rename(columns = {'Unnamed: 0': 'gene'}, inplace = True)
expressed_genes['gene'] = expressed_genes['gene'].apply(lambda x: int(x[1:])) 
expressed_genes.shape

(2351, 7)

In [21]:
expressed_genes = expressed_genes[expressed_genes.padj < 0.05]

In [22]:
expressed_genes

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,1,0.018163,5.042741,1.539113,23.447824,1.283478e-06,2.005860e-06
1,3,0.051592,18.278707,56.835503,41.162409,1.400911e-10,2.821639e-10
2,4,0.035982,4.806984,1.130483,31.574152,1.919679e-08,3.388779e-08
3,6,0.253950,-3.329484,0.470330,89.177551,3.609215e-21,1.239804e-20
4,10,0.015223,4.699783,1.789218,9.979771,1.582693e-03,1.975496e-03
...,...,...,...,...,...,...,...
2346,4993,2.666884,-4.063196,0.194795,774.204227,2.190995e-170,6.801968e-169
2347,4994,0.362137,-2.703292,0.361967,71.353284,2.986603e-17,8.534038e-17
2348,4995,0.038681,3.673674,0.801040,31.453368,2.042879e-08,3.597035e-08
2349,4996,0.018482,5.024842,1.582623,18.687107,1.540204e-05,2.252747e-05


In [23]:
expressed_genes = genes[expressed_genes.gene]

In [24]:
expressed_genes

array([b'HES4', b'TNFRSF18', b'TNFRSF4', ..., b'S100B', b'AC004556.1',
       b'AC233755.1'], dtype='|S19')